In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot  as plt

In [2]:
# constants
INPUT_HEIGHT = 640
INPUT_WIDTH = 640
SCORE_TRESHOLD = 0.5
NMS_THRESHOLD = 0.45
CONFIDENCE_THRESHOLD = 0.45
# text parametres
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1
# colors
BLACK = (0,0,0)
BLUE = (255,178,50)
YELLOW = (0,255,255)

In [3]:
image_path = "D:\Magistratura_3_semestr\id_446_value_271_545.jpg"
img = cv2.imread(image_path)

In [4]:
resized_img =  cv2.resize(img, (640, 640), interpolation=cv2.INTER_CUBIC)

In [5]:
cv2.imshow('sd', resized_img)
#cv2.imshow('output', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# functions

In [29]:
def get_crop(img, box):
    """
    функция для вырезания области индикаций в изображении
    """
    
    left, top, width, height = box
    crop_img = img[top-10: top+height+10, left-10:left+width+10]
    return crop_img

In [8]:
def pre_process(input_image, net):
    blob = cv2.dnn.blobFromImage(image=input_image, scalefactor=1/255, size=(INPUT_HEIGHT, INPUT_WIDTH), mean=[0,0,0], swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(net.getUnconnectedOutLayersNames())
    return outputs


In [49]:
def post_process(input_img, outputs):
    """
    функция выполнянет препроцессинг -> обработку предсказаний и отрисовку детекций на изображении
    """
    
    # lists to hold respective values while unwrapping
    class_ids = []
    confidences = []
    boxes = []
    indications_box = 0 
    # get image parametres
    image_height, image_width = input_img.shape[:2]
    x_factor = image_width / INPUT_WIDTH
    y_factor = image_height / INPUT_HEIGHT
    
    # drop garbage and transpose outputs
    outputs = outputs[0][0]
    outputs = outputs.transpose()
    # now we got outputs.shape = (8400, 6)

    # keep rows with scores > confidence_tereshold
    rows = [row for row in outputs if any(row[4:] > SCORE_TRESHOLD)]
    for row in rows:
        class_id = np.argmax(row[4:])
        confidences.append(row[4:][class_id])
        class_ids.append(class_id)
        xc, yc, w, h = row[:4]
        left = int((xc - w/2) * x_factor)
        top = int((yc - h/2) * y_factor)
        width = int(w * x_factor)
        height = int(h * y_factor)
        box = np.array([left, top, width, height])
        boxes.append(box)

    # perform nms
    indices = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_TRESHOLD, NMS_THRESHOLD)
    for i in indices:
        box = boxes[i]
        left, top, width, height = box
        print(box)
        # draw bbox
        cv2.rectangle(input_img, (left, top), (left + width, top + height), BLUE, 3*THICKNESS)

        # class labels
        # label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])

        # draw label
        # draw_label(input_img, label, left, top)
        # get box of indications
        if class_ids[i] == 1:
            indications_box = box
    # print(indications_box)  
    # if indications_box is None:
    #     return 0
    return indications_box

In [35]:
def resize_with_aspect_ratio(image, target_size=(640, 640), padding_color=(0, 0, 0)):
    """
    Изменяет размер изображения с сохранением пропорций и добавлением отступов.
    
    :param image: Исходное изображение (numpy array).
    :param target_size: Кортеж (width, height), к которому нужно привести изображение.
    :param padding_color: Цвет отступов (по умолчанию черный).
    :return: Измененное изображение с отступами.
    """
    # Исходные размеры
    h, w = image.shape[:2]
    target_width, target_height = target_size

    # Рассчитываем соотношения
    scale = min(target_width / w, target_height / h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    # Изменение размера с сохранением пропорций
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

    # Создаем пустое изображение с цветом фона
    result = np.full((target_height, target_width, 3), padding_color, dtype=np.uint8)

    # Вставляем измененное изображение в центр
    x_offset = (target_width - new_w) // 2
    y_offset = (target_height - new_h) // 2
    result[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_image

    return result

In [75]:
if __name__ == '__main__':
    
    image_folder = r'D:\Magistratura_3_semestr\Neironki\dataset_with_cropped_meters\not_cropped'
    output_folder = r'D:\Magistratura_3_semestr\Neironki\dataset_with_cropped_meters\croppped'

    classesFile = r"D:\Magistratura_3_semestr\Neironki\classes.txt"
    classes = None
    with open(classesFile, "rt") as f:
        classes = f.read().rstrip("\n").split("\n")
    cnt = 1
    for filename in os.listdir(image_folder):
        file_path = os.path.join(image_folder, filename)

        
    
        frame = cv2.imread(file_path)
        resized_frame =  cv2.resize(frame, (640, 640), interpolation=cv2.INTER_CUBIC)
        modelWeights = r"D:\Jupyter_Notebook\kgeu_abdullin\runs\train\yolov5n_exp14\weights\best.onnx"
        net = cv2.dnn.readNet(modelWeights)
    
        detections = pre_process(resized_frame, net)
        ind_box = post_process(resized_frame.copy(), detections)
        # print(type(ind_box))
        if isinstance(ind_box, np.ndarray):
            cropped_img = get_crop(resized_frame.copy(), ind_box) 
            resized_cropped_img = resize_with_aspect_ratio(cropped_img)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_cropped_img)
            # print('img saved')
        if cnt % 10 == 0:
            print(f'Saved {cnt}/125')
        cnt += 1
        
    # cv2.imshow('output', resized_cropped_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    

[ 46 114 523 373]
[114 219 298  52]
<class 'numpy.ndarray'>
[ 89 194 405 299]
[151 338 242  69]
<class 'numpy.ndarray'>
[ 56 156 551 402]
[155 371 310  63]
<class 'numpy.ndarray'>
[105 168 409 307]
[177 246 249  49]
<class 'numpy.ndarray'>
[ 99 180 471 330]
[192 261 271  55]
<class 'numpy.ndarray'>
[199 261 243  77]
[ 95 187 431 327]
<class 'numpy.ndarray'>
[ 15 164 622 476]
[147 290 406  82]
<class 'numpy.ndarray'>
[ 47 187 592 426]
[148 414 350  84]
<class 'numpy.ndarray'>
[ 28 224 531 399]
[119 315 305  51]
<class 'numpy.ndarray'>
[233 240 161  68]
[166 213 272 215]
<class 'numpy.ndarray'>
Saved 10/125
[ 15 142 533 280]
[132 211 308  42]
<class 'numpy.ndarray'>
[ 50 120 536 296]
[136 196 319  49]
<class 'numpy.ndarray'>
[  7  58 587 434]
[106 300 361  82]
<class 'numpy.ndarray'>
[ 43  87 587 454]
[185 230 322  64]
<class 'numpy.ndarray'>
[ 56 195 335 237]
[129 343 175  29]
<class 'numpy.ndarray'>
[ 92 127 545 302]
[199 219 321  46]
<class 'numpy.ndarray'>
[105 153 389 291]
[183 216 

In [62]:
image_folder = r'D:\Magistratura_3_semestr\Neironki\dataset_with_cropped_meters\not_cropped'
output_folder = r'D:\Magistratura_3_semestr\Neironki\dataset_with_cropped_meters\croppped'

In [64]:
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    test_img = cv2.imread(file_path)
    break
    print(output_path)
cv2.imshow('output', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
output_path = os.path.join(output_folder, "output_image.jpg")

# Сохранение изображения
cv2.imwrite(output_path, test_img)
print(f"Изображение сохранено по пути: {output_path}")

Изображение сохранено по пути: D:\Magistratura_3_semestr\Neironki\dataset_with_cropped_meters\croppped\output_image.jpg


In [73]:
len(os.listdir(image_folder))

125